In [ ]:
import cv2
import numpy as np
import os # Importar os para listar archivos
from sklearn.ensemble import RandomForestClassifier # Clasificador de ejemplo
from sklearn.metrics import mean_squared_error # Importar para calcular MSE

# from sklearn.svm import SVC # Otro clasificador de ejemplo

class OcularClassifier:
    def __init__(self, model_path=None):
        """
        Inicializa el clasificador.
        """
        self.classifier = RandomForestClassifier(random_state=42)
        if model_path:
            # En una aplicación real, cargarías tu modelo de clasificación entrenado aquí.
            # Para la demostración, entrenaremos uno simple más adelante.
            pass

    def _extract_features_from_image(self, image_path):
        """
        Args:
            image_path (str): Ruta a la imagen del ojo.

        Returns:
            list: Una lista de valores numéricos que representan las características extraídas.
                  [Alineación Ocular (0-4), Enrojecimiento Conjuntiva (0-4),
                   Claridad Córnea (0-4), Claridad Pupila/Cristalino (0-4),
                   Tamaño/Simetría Pupila (0/1), Secreciones (0-4),
                   Apariencia Párpados (0-3)]
                  Devuelve None si no se puede procesar la imagen.
        """
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: No se pudo leer la imagen en {image_path}")
            return None

        # Convertir a escala de grises para algunas operaciones
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Convertir a HSV para análisis de color
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # 1. Alineación Ocular (0-4) - SIMPLIFICADO AL EXTREMO
        # Esto requeriría detectar ambos ojos y medir su posición relativa.
        # Aquí, usamos una simulación basada en la posición de un punto central.
        rows, cols = gray.shape
        # Simulación: si el centro de la imagen no está cerca del centro esperado (muy básico)
        center_deviation = abs(rows // 2 - rows // 2) + abs(cols // 2 - cols // 2) # Siempre 0 en este caso, es solo ilustrativo
        alignment_score = min(4, center_deviation // 50) # Escala heurística. Reemplazar con lógica real.

        # 2. Enrojecimiento Conjuntiva (0-4) - SIMPLIFICADO (basado en análisis de color)
        # Detectar rangos de color rojo en el espacio HSV. Esto puede ser afectado por la iluminación.
        lower_red1 = np.array([0, 100, 100])
        upper_red1 = np.array([10, 255, 255])
        mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)

        lower_red2 = np.array([170, 100, 100])
        upper_red2 = np.array([180, 255, 255])
        mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)

        red_mask = mask_red1 + mask_red2
        # Calcular la proporción de píxeles rojos
        redness_proportion = np.sum(red_mask > 0) / (img.shape[0] * img.shape[1])
        # Escalar a una puntuación de 0 a 4 (esto es heurístico y no lineal)
        redness_score = min(4, int(redness_proportion * 10)) # Escala ajustada para demostración

        # 3. Claridad Córnea (0-4) - SIMPLIFICADO (basado en detección de bordes o texturas)
        # La claridad corneal real requeriría análisis de texturas finas o modelos específicos.
        # Aquí, usamos una métrica básica de variación local (varianza).
        # Una córnea clara debería tener menos variación local en una imagen bien enfocada.
        # Esto es muy sensible a la iluminación y el enfoque.
        clarity_score = 0 # Marcador de posición. La claridad real es compleja.
        # Una posible (pero básica) idea: calcular la varianza de los píxeles en una región de interés (ROI) de la córnea.
        # Cuanto menor sea la varianza, mayor podría ser la claridad (en condiciones ideales).
        kernel = np.ones((5,5),np.float32)/25
        dst = cv2.filter2D(gray,-1,kernel)
        variance = np.var(dst)
        clarity_score = min(4, int(5 - variance / 100)) # Escala inversa y heurística

        # 4. Claridad Pupila/Cristalino (0-4) - SIMPLIFICADO (basado en brillo/opacidad)
        # La opacidad del cristalino (cataratas) se manifestaría como una región
        # más clara u opaca dentro de la pupila.
        # Aquí, usamos una medida simple de brillo en la región central (asumiendo que es la pupila).
        pupil_clarity_score = 0 # Marcador de posición. La claridad real es compleja.
        # Una posible (pero básica) idea: calcular el brillo promedio o la varianza en una ROI de la pupila detectada.
        # Un brillo alto o mucha variación podría indicar opacidad.
        # (Necesitarías detectar la pupila primero)
        rows, cols = gray.shape
        center_row, center_col = rows // 2, cols // 2
        pupil_roi = gray[center_row - 20:center_row + 20, center_col - 20:center_col + 20]
        if pupil_roi.size > 0:
             pupil_brightness = np.mean(pupil_roi)
        #     # Una puntuación alta si el brillo es alto (indicando opacidad)
             pupil_clarity_score = min(4, int(pupil_brightness / 50))

        # 5. Tamaño/Simetría Pupila (0/1) - SIMPLIFICADO (basado en detección de círculos)
        # 0: Normal, 1: Anormal (diferencia significativa de tamaño o forma)
        # Usamos HoughCircles para una detección básica de círculos (pupilas).
        # Esto es muy sensible a la calidad de la imagen y los parámetros.
        pupil_symmetry_score = 0 # Asumimos normal por defecto
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20,
                                   param1=50, param2=30, minRadius=10, maxRadius=50)

        if circles is not None:
            circles = np.uint16(np.around(circles))
            # Si detectamos al menos dos círculos (posibles pupilas)
            if len(circles[0]) >= 2:
                # Tomamos los dos primeros círculos detectados como posibles pupilas
                pupil1 = circles[0][0]
                pupil2 = circles[0][1]

                radius1 = pupil1[2]
                radius2 = pupil2[2]

                # Calculamos la diferencia relativa de tamaño
                # Si la diferencia es mayor a un umbral (ej. 20%), consideramos asimetría
                if max(radius1, radius2) > 0 and abs(radius1 - radius2) / max(radius1, radius2) > 0.2:
                     pupil_symmetry_score = 1 # Anormal

            # Si solo se detecta un círculo o ninguno, no podemos determinar simetría
            # En un sistema real, manejarías estos casos de manera más robusta.


        # 6. Secreciones (0-4) - SIMPLIFICADO (basado en detección de regiones brillantes/blancas)
        # Las secreciones a menudo aparecen como acumulaciones blanquecinas o amarillentas.
        # Aquí, buscamos regiones muy brillantes en la imagen.
        # Esto es muy sensible a los reflejos.
        secretions_score = 0 # Marcador de posición. La detección real es compleja.
        # Una posible (pero básica) idea: buscar píxeles con valores altos en el canal V (Value) de HSV.
        lower_white = np.array([0, 0, 200])
        upper_white = np.array([255, 50, 255])
        mask_white = cv2.inRange(hsv, lower_white, upper_white)
        white_proportion = np.sum(mask_white > 0) / (img.shape[0] * img.shape[1])
        secretions_score = min(4, int(white_proportion * 20)) # Escala heurística

        # 7. Apariencia Párpados (0-3) - SIMPLIFICADO (basado en detección de bordes/texturas)
        # La inflamación, hinchazón o caída de párpados requeriría detección de los párpados
        # y análisis de su forma, textura y posición relativa al ojo.
        eyelid_score = 0 # Marcador de posición. La apariencia real es compleja.
        edges = cv2.Canny(gray, 100, 200)
        # (Necesitarías detectar los párpados primero para analizar solo esa región)
        eyelid_score = min(3, int(np.sum(edges) / 10000)) # Escala heurística

        features = [
            alignment_score,    # Alineación Ocular (0-4)
            redness_score,      # Enrojecimiento Conjuntiva (0-4)
            clarity_score,      # Claridad Córnea (0-4)
            pupil_clarity_score,# Claridad Pupila/Cristalino (0-4)
            pupil_symmetry_score,# Tamaño/Simetría Pupila (0/1)
            secretions_score,   # Secreciones (0-4)
            eyelid_score        # Apariencia Párpados (0-3)
        ]

        return features

    def train_classifier(self, image_folder, labels_dict):
        """
        Extrae características de imágenes en una carpeta y entrena el modelo de clasificación.
        Args:
            image_folder (str): Ruta a la carpeta que contiene las imágenes de entrenamiento.
            labels_dict (dict): Diccionario mapeando nombres de archivo a etiquetas (0-5).
        Returns:
            tuple: (X_train, y_train) numpy arrays used for training.
        """
        X_train = []
        y_train = []
        print(f"\nCargando imágenes de {image_folder} para entrenamiento...")
        for filename in os.listdir(image_folder):
            if filename.endswith(('.jpg', '.png', '.jpeg')): # Filtrar por extensiones de imagen
                image_path = os.path.join(image_folder, filename)
                features = self._extract_features_from_image(image_path)
                if features is not None and filename in labels_dict:
                    X_train.append(features)
                    y_train.append(labels_dict[filename])
                elif filename not in labels_dict:
                    print(f"Advertencia: No se encontró etiqueta para {filename}. Se omitirá.")

        if not X_train:
            print("Error: No se encontraron imágenes válidas con etiquetas para entrenar.")
            return None, None

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        print("\nEntrenando modelo de clasificación...")
        self.classifier.fit(X_train, y_train)
        print("Modelo entrenado.")
        return X_train, y_train

    def predict_visual_problem(self, image_path):
        """
        Extrae características de una imagen y predice el problema visual.
        Args:
            image_path (str): Ruta a la imagen de entrada.

        Returns:
            str: Problema visual predicho o sugerencia de consulta especializada.
        """
        features = self._extract_features_from_image(image_path)
        if features is None:
            return "Error en la extracción de características de la imagen."

        # Convertir las características a un array numpy para la predicción
        features_array = np.array(features).reshape(1, -1)

        # Predecir usando el clasificador entrenado
        prediction_index = self.classifier.predict(features_array)[0]
        # Mapear el índice de vuelta a la etiqueta de cadena
        # Esto asume que tus etiquetas de entrenamiento corresponden a estos índices
        # 0: Visión Normal, 1: Estrabismo, 2: Conjuntivitis, 3: Cataratas, 4: Ojo Seco
        possible_problems = {
            0: "Visión Normal",
            1: "Estrabismo",
            2: "Conjuntivitis",
            3: "Cataratas",
            4: "Ojo Seco",
            5: "Consulta especializada" # Mapeo para el caso de "Consulta especializada"
        }
        predicted_problem = possible_problems.get(prediction_index, "Consulta especializada")

        # Por ejemplo, si ciertas características son altamente indicativas de una condición específica
        if features[0] >= 3: # Puntuación alta de alineación (simulando desalineación severa)
             return "Estrabismo" # Anular basándose en una regla fuerte
        if features[1] >= 3: # Puntuación alta de enrojecimiento
             return "Conjuntivitis"
        if features[3] >= 3: # Opacidad alta en pupila/cristalino
             return "Cataratas"
        if features[5] >= 2 and features[6] >= 2: # Secreciones y problemas en los párpados
             return "Ojo Seco"

        # Regla para sugerir "Consulta especializada" si el modelo clasifica con un índice 5
        # o si las características son ambiguas (definidas en la función suggest_specialized_consultation)
        if prediction_index == 5 or self.suggest_specialized_consultation(features):
            return "Consulta especializada"

        return predicted_problem

    def suggest_specialized_consultation(self, features):
        """
        Una función basada en reglas para sugerir consulta especializada
        si las características no se alinean claramente con condiciones conocidas o son ambiguas.
        """
        if any(f > 1 for f in features) and sum(features) < 5: # Algunos problemas, pero no graves o específicos
            return True
        return False

# --- Ejemplo de Uso ---
# 1. Instanciar el clasificador
ocular_classifier = OcularClassifier()

# 2. Preparar DATOS DE ENTRENAMIENTO
# Debes tener una carpeta con imágenes de entrenamiento y un diccionario con sus etiquetas.
image_folder = '/content/Entrenamiento'
entrenamiento_path = '/content/Entrenamiento'
all_entries = os.listdir(entrenamiento_path)
files_in_entrenamiento = [entry for entry in all_entries if os.path.isfile(os.path.join(entrenamiento_path, entry))]

labels_dict = {}

for file_name in files_in_entrenamiento:
    if file_name.startswith('normal'):
        labels_dict[file_name] = 0
    elif file_name.startswith('estrabismo'):
        labels_dict[file_name] = 1
    elif file_name.startswith('conjuntivitis'):
        labels_dict[file_name] = 2
    elif file_name.startswith('cataratas'):
        labels_dict[file_name] = 3
    elif file_name.startswith('ojo_seco'):
        labels_dict[file_name] = 4
# Puedes agregar un caso para archivos que no coincidan con ningún prefijo,
# o asumir que no son relevantes para el entrenamiento.
# Puedes acceder al diccionario completo con la variable 'labels_dict'

# Entrenar el clasificador
X_train, y_train = ocular_classifier.train_classifier(image_folder, labels_dict)


# 3. Probar con todas las imágenes en la carpeta Prediccion
print("\n--- Probando predicción con todas las imágenes en la carpeta Prediccion ---")
prediction_folder = '/content/Prediccion'

if os.path.exists(prediction_folder):
# List all entries in the prediction folder
  all_prediction_entries = os.listdir(prediction_folder)
# Filter out entries that are directories, leaving only files
  prediction_image_files = [entry for entry in all_prediction_entries if os.path.isfile(os.path.join(prediction_folder, entry))]

  if prediction_image_files:
      for image_file in prediction_image_files:
          test_image_path = os.path.join(prediction_folder, image_file)
          prediction_test = ocular_classifier.predict_visual_problem(test_image_path)
          print(f"Predicción para '{image_file}': {prediction_test}")
  else:
      print(f"\nAdvertencia: No se encontraron archivos de imagen en '{prediction_folder}'.")
else:
  print(f"\nAdvertencia: No se encontró la carpeta de predicción en '{prediction_folder}'.")